## Nueva filosofía

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, input_file_name
from pyspark.sql.types import StringType
import re

In [2]:
spark = (
    SparkSession.builder
    .appName("CargarPartidas")
    .master("local[*]")  # "yarn"
    .getOrCreate()
)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 17:50:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Por si se quiere eliminar alguna carpeta
!hdfs dfs -rm -r /user/ajedrez/jugador
# En principio, solo con la carpeta raw y las partidas dentro nos vale
!hdfs dfs -ls /user/ajedrez/

Deleted /user/ajedrez/jugador
Found 1 items
drwxr-xr-x   - root supergroup          0 2025-05-09 11:46 /user/ajedrez/raw


In [4]:
# Leer todos los archivos como (ruta, contenido)
rdd = spark.sparkContext.wholeTextFiles("hdfs:///user/ajedrez/raw/*.pgn")

# Dividir cada archivo en partidas
def dividir_partidas_por_archivo(nombre_y_contenido):
    ruta, contenido = nombre_y_contenido
    partidas = re.split(r'\n(?=\[Event )', contenido)
    return [(ruta, pgn) for pgn in partidas if "[Event" in pgn]

rdd_partidas = rdd.flatMap(dividir_partidas_por_archivo)

# Convertir a DataFrame
df = rdd_partidas.toDF(["archivo", "pgn"])

# UDFs para extraer tags
def extraer_tag(tag, texto):
    match = re.search(rf'\[{tag} "([^"]+)"\]', texto)
    return match.group(1) if match else None

extraer_white = udf(lambda x: extraer_tag("White", x), StringType())
extraer_black = udf(lambda x: extraer_tag("Black", x), StringType())
extraer_date = udf(lambda x: extraer_tag("Date", x), StringType())

df = df.withColumn("white", extraer_white(col("pgn"))) \
       .withColumn("black", extraer_black(col("pgn"))) \
       .withColumn("date", extraer_date(col("pgn")))


# Partidas donde Nakamura juega con blancas
nakamura_white_df = df.filter(col("white") == "Nakamura,Hi")

# Partidas donde Nakamura juega con negras
nakamura_black_df = df.filter(col("black") == "Nakamura,Hi")

# Guardar en HDFS
# nakamura_white_df.select("pgn").write.mode("overwrite").text("hdfs:///user/ajedrez/jugador/Nakamura_blancas")
# nakamura_black_df.select("pgn").write.mode("overwrite").text("hdfs:///user/ajedrez/jugador/Nakamura_negras")

In [ ]:
# En principio, solo con la carpeta raw y las partidas dentro nos vale
# !hdfs dfs -ls /user/ajedrez/jugador/

In [6]:
filas = nakamura_white_df.count()
columnas = len(nakamura_white_df.columns)
print(f"Shape: ({filas}, {columnas})")

print(nakamura_white_df.columns)


Shape: (35, 5)
['archivo', 'pgn', 'white', 'black', 'date']


In [7]:
nakamura_white_df.select("white", "black", "date").show(truncate=False, n=10)

+-----------+----------------+----------+
|white      |black           |date      |
+-----------+----------------+----------+
|Nakamura,Hi|Vidit,S         |2024.04.05|
|Nakamura,Hi|Praggnanandhaa,R|2024.04.07|
|Nakamura,Hi|Nevednichy,V    |2024.04.02|
|Nakamura,Hi|Eljanov,P       |2024.04.02|
|Nakamura,Hi|Kuzubov,Y       |2024.04.02|
|Nakamura,Hi|Kovalev,Vl      |2024.04.02|
|Nakamura,Hi|Duda,J          |2024.04.02|
|Nakamura,Hi|Makarian,Rudik  |2024.04.02|
|Nakamura,Hi|Richter,Paul    |2024.04.02|
|Nakamura,Hi|Durarbayli,Vasif|2024.04.02|
+-----------+----------------+----------+
only showing top 10 rows



In [ ]:
print(df.select("pgn").first()["pgn"])

### Se puede arreglar eso de separar en dos dataframes para luego unirlos en 1 es mejor no separarlos

In [21]:
from pyspark.sql.functions import udf, explode, col
from pyspark.sql.types import ArrayType, StructType, StructField, StringType
import chess.pgn
import io

# UDF para extraer jugadas (FEN, movimiento, color)
def pgn_a_jugadas(pgn_str, color):
    resultado = []
    try:
        game = chess.pgn.read_game(io.StringIO(pgn_str))
        if not game:
            return []

        board = game.board()
        color_jugador = chess.WHITE if color == "white" else chess.BLACK

        for move in game.mainline_moves():
            if board.turn == color_jugador:
                resultado.append((board.fen(), move.uci(), color))
            board.push(move)
    except:
        pass
    return resultado

# Definir esquema
esquema_jugada = ArrayType(StructType([
    StructField("FEN", StringType(), True),
    StructField("Move", StringType(), True),
    StructField("Color", StringType(), True)
]))

# Registrar UDF (parámetro adicional: color)
pgn_udf = udf(lambda pgn: pgn_a_jugadas(pgn, "white"), esquema_jugada)
pgn_udf_black = udf(lambda pgn: pgn_a_jugadas(pgn, "black"), esquema_jugada)

# Aplicar UDF y explotar jugadas
df_jugadas_blancas = nakamura_white_df.withColumn("jugadas", explode(pgn_udf(col("pgn")))) \
    .select(col("jugadas.FEN"), col("jugadas.Move"), col("jugadas.Color"))

df_jugadas_negras = nakamura_black_df.withColumn("jugadas", explode(pgn_udf_black(col("pgn")))) \
    .select(col("jugadas.FEN"), col("jugadas.Move"), col("jugadas.Color"))

# Unir ambos conjuntos
# df_jugadas = df_jugadas_blancas.union(df_jugadas_negras)

# Cachear para uso posterior con MLlib
df_jugadas_blancas.cache()
df_jugadas_blancas.show(truncate=False)

df_jugadas_negras.cache()
df_jugadas_negras.show(truncate=False)


+--------------------------------------------------------------------+----+-----+
|FEN                                                                 |Move|Color|
+--------------------------------------------------------------------+----+-----+
|rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1            |e2e4|white|
|rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2        |g1f3|white|
|r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3    |f1b5|white|
|r1bqkb1r/pppp1ppp/2n2n2/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 4 4 |d2d3|white|
|r1bqk2r/pppp1ppp/2n2n2/1Bb1p3/4P3/3P1N2/PPP2PPP/RNBQK2R w KQkq - 1 5|c2c3|white|
|r1bq1rk1/pppp1ppp/2n2n2/1Bb1p3/4P3/2PP1N2/PP3PPP/RNBQK2R w KQ - 1 6 |e1g1|white|
|r1bq1rk1/ppp2ppp/2np1n2/1Bb1p3/4P3/2PP1N2/PP3PPP/RNBQ1RK1 w - - 0 7 |h2h3|white|
|r1bq1rk1/ppp1nppp/3p1n2/1Bb1p3/4P3/2PP1N1P/PP3PP1/RNBQ1RK1 w - - 1 8|d3d4|white|
|r1bq1rk1/pp2nppp/2pp1n2/1Bb1p3/3PP3/2P2N1P/PP3PP1/RNBQ1RK1 w - - 0 9|b5d3|white|
|r1bq1rk1/pp2npp

⚠️ Consideraciones clave para adaptar tu flujo a Spark MLlib:
MLlib no soporta tensores 3D (8x8x12) directamente.

MLlib trabaja con Vector (denso o disperso), por lo tanto debes aplanar tu tensor a un vector de tamaño 8×8×12 = 768.

No hay soporte nativo para LabelEncoder de sklearn, pero puedes hacer lo mismo con StringIndexer de Spark.

UDFs con NumPy son posibles, pero deben devolver estructuras planas (List[Float]), no arrays 3D.

In [10]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, DoubleType, IntegerType
import numpy as np

# Mapas globales para codificar piezas y movimientos
piece_to_plane = {
    'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
    'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
}

# Codificar el tablero como vector 1D (768 elementos)
def fen_to_vector(fen):
    tensor = np.zeros((8, 8, 12), dtype=np.float32)
    fen_board = fen.split(' ')[0]
    rows = fen_board.split('/')
    
    for i, row in enumerate(rows):
        col = 0
        for char in row:
            if char.isdigit():
                col += int(char)
            elif char in piece_to_plane:
                plane = piece_to_plane[char]
                tensor[i, col, plane] = 1
                col += 1
    return tensor.flatten().tolist()

# Registrar como UDF en Spark
fen_udf = udf(fen_to_vector, ArrayType(FloatType()))
df_feat = df_jugadas.withColumn("features", fen_udf(col("FEN")))


In [11]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Move", outputCol="label")
df_final = indexer.fit(df_feat).transform(df_feat).select("features", "label")

Lo que puedes hacer en entorno distribuido con Spark MLlib:
MLlib no soporta CNNs ni tensores 4D. Su diseño está enfocado a:

Modelos clásicos: árboles, regresión, SVM, redes densas básicas.

Operación sobre vectores 1D (features) y etiquetas (label).

Entrenamiento distribuido y escalable, pero no deep learning como en Keras.

2. Alternativas si necesitas CNN en un entorno distribuido
Aunque MLlib no soporta CNN, puedes usar frameworks especializados en deep learning que sí funcionan en entornos distribuidos:

a) TensorFlow o PyTorch con Horovod o Spark
Puedes integrar TensorFlow o PyTorch con Horovod, que permite entrenamiento distribuido sobre múltiples nodos.

También puedes usar TensorFlowOnSpark o BigDL (una biblioteca de deep learning optimizada para Spark) para ejecutar redes neuronales sobre clústeres.

b) BigDL
BigDL es una alternativa poderosa si ya estás usando Spark y quieres hacer deep learning directamente sobre ese ecosistema.

Soporta CNN, RNN y otros modelos complejos, y permite entrenamiento distribuido.

Conclusión
MLlib no es adecuado para CNN, pero puedes usar herramientas externas como TensorFlow + Horovod, BigDL o TensorFlowOnSpark si necesitas entrenamiento distribuido de redes neuronales profundas.

⚠️ Limitación clave: Spark ≠ TensorFlow
Spark y TensorFlow usan entornos de ejecución diferentes:

Spark distribuye y mantiene df_final en su propio contexto de ejecución (con sus RDDs y DAGs).

TensorFlow o Keras necesita los datos en forma de NumPy o Tensor en memoria local.

In [12]:
df_final.printSchema()
df_final.show(1)

root
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- label: double (nullable = false)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0, 0.0, 0.0, 0...| 29.0|
+--------------------+-----+
only showing top 1 row



In [13]:
# Guardar DataFrame como Parquet en volumen compartido
# df_final.write.mode("overwrite").parquet("/shared/fen_jugadas.parquet")   NO FUNCIONA
df_pandas = df_final.toPandas()
df_pandas.to_parquet("/shared/fen_jugadas.parquet")

In [16]:
!ls -l /shared/fen_jugadas.parquet

-rw-r--r-- 1 root root 84626 May 10 18:02 /shared/fen_jugadas.parquet


In [18]:
# ENTRENAMIENTO DISTRIBUIDO

from tensorflowonspark import TFCluster, TFNode
from pyspark.sql import SparkSession
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Parámetros
data_path = "/shared/fen_jugadas.parquet"
model_path = "/shared/modelo_distribuido"

def model_fn():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(8, 8, 12)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def main_fun(args, ctx):
    df = pd.read_parquet(args["data_path"])
    X = np.array(df["features"].tolist()).reshape(-1, 8, 8, 12)
    y = df["label"].astype(np.int32).values

    global NUM_CLASSES
    NUM_CLASSES = len(np.unique(y))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    model = model_fn()
    model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=32)

    if ctx.job_name == "chief":
        model.save(args["model_path"])

# Iniciar Spark y lanzar el cluster
spark = SparkSession.builder.appName("TFoS-CNN").getOrCreate()
sc = spark.sparkContext

args = {"data_path": data_path, "model_path": model_path}

cluster = TFCluster.run(sc, main_fun, args, num_executors=2, num_ps=1, input_mode=TFCluster.InputMode.TENSORFLOW, master_node="chief", log_dir="/tmp/tf_logs")
cluster.shutdown()


25/05/10 18:07:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2025-05-10 18:07:55.153280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 18:07:55.153276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 18:07:55.371556: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 18:07:55.371770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-10 18:07:55.372114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-10 18:07:55.3721

In [40]:
## ENTRENAMIENTO LOCAL EN SPARK-CLIENT (USÉ EL ANTERIOR)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Cargar datos exportados desde Spark
df = pd.read_parquet("/shared/fen_jugadas.parquet")

# Convertir a tensores NumPy
X = np.array(df["features"].tolist()).reshape(-1, 8, 8, 12)
y = df["label"].values.astype(int)

# Dividir en entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Definir el modelo CNN
model = Sequential([
    Input(shape=(8, 8, 12)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')
])

# Compilar y entrenar
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Guardar modelo
model.save("/shared/modelo_nakamura.keras")
np.save("/shared/encoder.npy", np.unique(y))  # Etiquetas numéricas


Epoch 1/10


2025-05-10 11:37:23.932746: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0071 - loss: 6.8118 - val_accuracy: 0.0031 - val_loss: 6.4299
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0148 - loss: 6.3681 - val_accuracy: 0.0125 - val_loss: 6.4215
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0171 - loss: 6.2490 - val_accuracy: 0.0094 - val_loss: 6.4129
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0202 - loss: 6.0973 - val_accuracy: 0.0250 - val_loss: 6.4572
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0246 - loss: 5.9505 - val_accuracy: 0.0219 - val_loss: 6.5551
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0329 - loss: 5.8137 - val_accuracy: 0.0312 - val_loss: 6.5765
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0437 - loss: 5.6680 - val_accuracy: 0.0344 - val_loss: 6.6006
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0515 - loss: 5.3731 - val_accuracy: 0.0312 - val_loss: 7.

In [14]:
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, IntegerType
from pyspark.ml.feature import StringIndexer

# Codificar FEN como vector plano 768 (8x8x12)
def fen_to_flat_vector(fen):  
    piece_to_plane = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    
    tensor = np.zeros((8, 8, 12), dtype=np.float32)
    fen_board = fen.split(' ')[0]
    rows = fen_board.split('/')
    
    for i, row in enumerate(rows):
        col = 0
        for char in row:
            if char.isdigit():
                col += int(char)
            elif char in piece_to_plane:
                plane = piece_to_plane[char]
                tensor[i, col, plane] = 1
                col += 1
    return tensor.flatten().tolist()

# UDF para usar en Spark
fen_udf = udf(fen_to_flat_vector, ArrayType(FloatType()))

# Aplicar UDF para obtener features
df_ml = df_jugadas.withColumn("features", fen_udf(col("FEN")))

# Codificar movimiento como etiqueta
indexer = StringIndexer(inputCol="Move", outputCol="label")
indexer_model = indexer.fit(df_ml)
df_ml = indexer_model.transform(df_ml)

# Seleccionar columnas relevantes para MLlib
df_final = df_ml.select("features", "label")

# Mostrar ejemplo
# df_final.show(3, truncate=False)

print(df_final.columns)


['features', 'label']
